In [1]:
import json
import logging
import asyncio
import pprint

import pandas as pd

from llm_plotting.settings import Settings
from llm_plotting.prompt_helper import extract_metadata, setup_agent_executor, pull_image_in_base64_history

if __name__ == "__main__":
    logging.basicConfig(level=logging.INFO)

    # ==================== User setup/input ====================
    settings = Settings()
    user_input = "make a plot of the average salary of every job, and validate it"

    df = pd.read_csv("jobs_in_data.csv")
    metadata_json = extract_metadata(df)

    # ==================== Agent setup ====================

    agent_executor = setup_agent_executor(settings, df)



In [13]:
a = """
You are a powerful code generation assistant who specializes in generating code \
to visualize graphs. You can also generate code to answer questions about user \
DataFrames (always stored under df.csv). When answering questions about the data, \
you must always send your answer in a print statement."""

In [14]:
print(a)


You are a powerful code generation assistant who specializes in generating code to visualize graphs. You can also generate code to answer questions about user DataFrames (always stored under df.csv). When answering questions about the data, you must always send your answer in a print statement.


In [3]:
import pandas as pd

In [4]:
df = pd.read_csv("jobs_in_data.csv")

In [11]:
df.loc[0,'salary'] = 'a'

/tmp/ipykernel_7738/3389270308.py:1: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'a' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.



In [12]:
import pandas as pd
import plotly.express as px

avg_salary_per_job = df.groupby('job_title')['salary'].mean().reset_index()

# Selecting top 20 job titles by average salary for better legibility
avg_salary_per_job = avg_salary_per_job.sort_values(by='salary', ascending=False).head(20)

fig = px.bar(avg_salary_per_job, x='job_title', y='salary', title='Top 20 Job Titles by Average Salary')
fig.update_layout(xaxis_tickangle=-45, xaxis_title='Job Title', yaxis_title='Average Salary (USD)',
                  title={'x':0.5, 'xanchor': 'center'}, uniformtext_minsize=8, uniformtext_mode='hide')
fig.update_traces(marker_color='rgb(158,202,225)', marker_line_color='rgb(8,48,107)',
                  marker_line_width=1.5, opacity=0.6)
fig.show()

TypeError: agg function failed [how->mean,dtype->object]

In [1]:
def append_to(num, target=[]):
    target.append(num)
    return target

print(append_to(1))  # prints: [1]
print(append_to(2))  # prints: [1, 2]
print(append_to(3))  # prints: [1, 2, 3]

[1]
[1, 2]
[1, 2, 3]


In [8]:
def parse_requirements(filename, names_only=False):
    with open(filename, 'r') as file:
        lines = file.readlines()

    result = ""
    for line in lines:
        line = line.strip()
        if not line:  # skip empty lines
            continue
        if "==" in line:
            package, version = line.split("==")
            if names_only:
                result += f"{package}, "
            else:
                result += f"{package} = \"{version}\"\n"
        elif ">=" in line:
            package, version = line.split(">=")
            if names_only:
                result += f"{package}, "
            else:
                result += f"{package} = \"^{version}.0\"\n"
        elif "~=" in line:
            package, version = line.split("~=")
            if names_only:
                result += f"{package}, "
            else:
                result += f"{package} = \"^{version}\"\n"
        else:
            if names_only:
                result += f"{line}, "
            else:
                result += f"{line} = \"^\"\n"

    return result.rstrip(', ') if names_only else result

print(parse_requirements("llm_plotting/e2b/requirments.txt"))

plotly = "^5.18"
pandas = "^2.2"
numpy = "^1.26"
kaleido = "0.2.1"



In [1]:
from llm_plotting.settings import Settings
settings = Settings()

In [1]:
from llm_plotting.parsers import CodeAndDescription
from langchain.output_parsers import PydanticOutputParser

code_generation_output_parser = PydanticOutputParser(pydantic_object=CodeAndDescription)

In [2]:
code_generation_output_parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"code": {"title": "Code", "description": "code to generate the plots", "type": "string"}, "description": {"title": "Description", "description": "brief description of the code and plots", "type": "string"}}, "required": ["code", "description"]}\n```'

In [2]:
from e2b import Sandbox


In [3]:
sandbox = Sandbox(template="my-agent-sandbox-test", api_key=settings.e2b_api_key)

In [9]:
process = sandbox.process.start_and_wait('df -m')
print(process.stdout)
print(process.stderr)

Filesystem     1M-blocks  Used Available Use% Mounted on
/dev/root           2001   997       988  51% /
devtmpfs             237     0       237   0% /dev
tmpfs                239     0       239   0% /dev/shm
tmpfs                 96     1        96   1% /run
tmpfs                  5     0         5   0% /run/lock



In [3]:
async def run_agent():
    chunks = []

    async for chunk in agent_executor.astream(
        {"user_input": user_input, "metadata_json": metadata_json}
    ):
        chunks.append(chunk)
        print("------")
        pprint.pprint(chunk, depth=1)

    return chunks 
chunks = asyncio.run(run_agent())

RuntimeError: asyncio.run() cannot be called from a running event loop

In [4]:
chuncks = asyncio.run(run_agent())

RuntimeError: asyncio.run() cannot be called from a running event loop

In [11]:
a = (
    "import pandas as pd\n"
    "import plotly.express as px\n"
    "import plotly.io as pio\n\n"
    "# Load the data\n"
    "df = pd.read_csv('df.csv')\n\n"
    "# Calculate the average salary for each job\n"
    "avg_salary = df.groupby('job_title')['salary'].mean().reset_index()\n\n"
    "# Create the plot\n"
    "fig = px.bar(avg_salary, x='job_title', y='salary', title='Average Salary by Job')\n\n"
    "# Save the figure\n"
    "pio.write_image(fig, 'figure.png')"
)
print(a)

import pandas as pd
import plotly.express as px
import plotly.io as pio

# Load the data
df = pd.read_csv('df.csv')

# Calculate the average salary for each job
avg_salary = df.groupby('job_title')['salary'].mean().reset_index()

# Create the plot
fig = px.bar(avg_salary, x='job_title', y='salary', title='Average Salary by Job')

# Save the figure
pio.write_image(fig, 'figure.png')


In [2]:
#==================== Run agent ====================
output = agent_executor.invoke(
    {"user_input": user_input, "metadata_json": metadata_json}
)




> Entering new AgentExecutor chain...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Could not parse tool input: {'arguments': "import pandas as pd\nimport plotly.express as px\nimport plotly.io as pio\n\n# Load the data\ndf = pd.read_csv('df.csv')\n\n# Calculate the average salary for each job\navg_salary = df.groupby('job_title')['salary'].mean().reset_index()\n\n# Create the plot\nfig = px.bar(avg_salary, x='job_title', y='salary', title='Average Salary by Job')\n\n# Save the figure\npio.write_image(fig, 'figure.png')", 'name': 'python'} because the `arguments` is not valid JSON.Invalid or incomplete response

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:e2b.sandbox.main:Creating sandbox my-agent-sandbox-test
INFO:e2b.sandbox.sandbox_connection:Sandbox for template my-agent-sandbox-test initialized
INFO:e2b.sandbox.main:Opening sandbox my-agent-sandbox-test



Invoking: `CodeValidationTool` with `{'code': "import pandas as pd\nimport plotly.express as px\nimport plotly.io as pio\n\n# Load the data\ndf = pd.read_csv('df.csv')\n\n# Calculate the average salary for each job\navg_salary = df.groupby('job_title')['salary'].mean().reset_index()\n\n# Create the plot\nfig = px.bar(avg_salary, x='job_title', y='salary', title='Average Salary by Job')\n\n# Save the figure\npio.write_image(fig, 'figure.png')", 'description': 'Plot of the average salary of every job'}`




...

There is an issue with the plot provided: the x-axis labels (job titles) are overlapping and unreadable. This makes it difficult to associate the individual salary values with their corresponding job titles accurately.

To fix this issue, you can try one or more of the following:
1. Rotate the x-axis labels to make them vertical or at a 45-degree angle to prevent overlap.
2. Increase the width of the plot to spread out the labels.
3. Decrease the font size slightly for the axis labels.
4. If the number of categories (job titles) is too high, consider using an interactive plot that allows users to zoom in on the axis or hover over bars for detailed information.

Please regenerate the code with the suggested changes and run the code through this validation tool before returning the code to the user.

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:e2b.sandbox.main:Creating sandbox my-agent-sandbox-test
INFO:e2b.sandbox.sandbox_connection:Sandbox for template my-agent-sandbox-test initialized
INFO:e2b.sandbox.main:Opening sandbox my-agent-sandbox-test



Invoking: `CodeValidationTool` with `{'code': "import pandas as pd\nimport plotly.express as px\nimport plotly.io as pio\n\n# Load the data\ndf = pd.read_csv('df.csv')\n\n# Calculate the average salary for each job\navg_salary = df.groupby('job_title')['salary'].mean().reset_index()\n\n# Create the plot\nfig = px.bar(avg_salary, x='job_title', y='salary', title='Average Salary by Job', color='job_title')\n\n# Customize the plot\nfig.update_layout(xaxis_tickangle=-45, xaxis_tickfont=dict(size=10), width=800, height=500, plot_bgcolor='white', paper_bgcolor='white')\n\n# Save the figure\npio.write_image(fig, 'figure.png')", 'description': 'Plot of the average salary of every job'}`




INFO:e2b.sandbox.sandbox_connection:Sandbox tn3bvfw713dxzl0l798c created (id:ivj7kg1q8b7xd7erxwwb5)
INFO:e2b.sandbox.sandbox_connection:Started refreshing sandbox tn3bvfw713dxzl0l798c (id: ivj7kg1q8b7xd7erxwwb5)
INFO:e2b.sandbox.sandbox_rpc:WebSocket waiting to start
INFO:e2b.sandbox.websocket_client:WebSocket connected to wss://49982-ivj7kg1q8b7xd7erxwwb5-e645bfea.e2b.dev/ws
INFO:e2b.sandbox.sandbox_rpc:WebSocket started
INFO:e2b.sandbox.main:Sandbox my-agent-sandbox-test opened
INFO:e2b.sandbox.process:Starting process: python3 /index.py
INFO:e2b.sandbox.process:Started process (id: OsPkpXIbg3dO)
INFO:e2b.sandbox.process:Process OsPkpXIbg3dO exited with exit code 0
INFO:e2b.sandbox.process:Handling process exit (id: OsPkpXIbg3dO)
INFO:e2b.sandbox.sandbox_connection:Closing sandbox tn3bvfw713dxzl0l798c (id: ivj7kg1q8b7xd7erxwwb5)
INFO:e2b.sandbox.sandbox_connection:Sandbox closed
INFO:e2b.sandbox.websocket_client:WebSocket stopped
INFO:e2b.sandbox.sandbox_connection:Stopped refreshing

The plot is not legible due to the following issues:

1. The x-axis labels (job titles) are overlapping and unreadable. This makes it impossible to distinguish between the different job titles.
2. The plot has too many categories close together, which makes it difficult to see individual bars clearly and to compare between them.

Here are some suggestions to improve the plot:

- Consider using a larger plot size or smaller font for the job titles to avoid overlapping.
- A more effective visualization could be to select a subset of the data, such as the top 10 highest-paid jobs or grouping similar job titles.
- Another solution is to rotate the job title text or utilize interactive hovering features that display the job title when hovering over a specific bar.
- If there's a need to show all the data, then an interactive plot where users can zoom in and scroll could be beneficial.
- Use different types of charts, such as a horizontal bar chart, which might accommodate the job titles bet